**DYNAMIC PROTEIN LOCALIZATION**

In [4]:
import numpy as np
import pandas as pd

In [109]:
file_path = 'StaticData.txt'
# Column names initialisation
vec = [
    "orfid", "yORF", "gene name", "GFP tagged", "GFP visualized?", "TAP visualized?", 
    "abundance", "error", "localization summary", "ambiguous", "mitochondrion", 
    "vacuole", "spindle pole", "cell periphery", "punctate composite", 
    "vacuolar membrane", "ER", "nuclear periphery", "endosome", "bud neck", 
    "microtubule", "Golgi", "late Golgi", "peroxisome", "actin", "nucleolus", 
    "cytoplasm", "ER to Golgi", "early Golgi", "lipid particle", "nucleus", "bud"
]

# Load dataset
static_data = pd.read_csv(file_path, delimiter='\t', header=None, index_col=None, skipinitialspace=True)
# Last two columns are tabs
static_data = static_data.iloc[:, :-2]
# Set column names
static_data.columns = vec

We start working on the STATIC DATA

In [110]:
# Check dataset
static_data

,orfid,yORF,gene name,GFP tagged,GFP visualized?,TAP visualized?,abundance,error,localization summary,ambiguous,...,late Golgi,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud
0,206,YAL001C,TFC3,tagged,visualized,TAP visualized,125,NaN,"cytoplasm,nucleus",F,...,F,F,F,F,T,F,F,F,T,F
1,5281,YAL002W,VPS8,tagged,visualized,TAP visualized,736,NaN,endosome,F,...,F,F,F,F,F,F,F,F,F,F
2,1073,YAL005C,SSA1,tagged,visualized,TAP visualized,269000,NaN,cytoplasm,F,...,F,F,F,F,T,F,F,F,F,F
3,6096,YAL007C,ERP2,tagged,visualized,TAP visualized,26300,NaN,ER,F,...,F,F,F,F,F,F,F,F,F,F
4,3882,YAL008W,FUN14,tagged,visualized,TAP visualized,1920,NaN,mitochondrion,F,...,F,F,F,F,F,F,F,F,F,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,4500,YPR200C,ARR2,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,F
6230,5814,YPR201W,ARR3,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,F
6231,3172,YPR202W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,F
6232,5012,YPR203W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,F


In [111]:
# TO BE ASKED: what is error column? Is it usefull? (lot of nans in it)
print(np.sum(static_data["error"].isna()))
# TO BE ASKED: We have 2078 proteins untracked....
print(np.sum(static_data["localization summary"].isna()))

6028
2078


CHIEDERE:

- Quando l'errore è nan, cioè cosa rappresenta

- Cosa significano alcune colonne del dataset statico


In [112]:
# We export the table as csv
# DA SPOSTARE POST PULIZIA DATASET
output_file_path = 'static_data.csv'
static_data.to_csv(output_file_path, index=False)

In [113]:
# Handle fasta file with protein sequences and their identifier
def parse_fasta(file_path):
    results = []

    with open(file_path, 'r') as file:
        identifier = None
        sequence = ""
        
        for line in file:
            line = line.strip()
            
            if line.startswith(">"):
                # Salva i dati della sequenza precedente se ci sono
                if identifier and sequence:
                    results.append((identifier, sequence))
                
                # Reset per la nuova sequenza
                identifier = line.split()[0][1:]  # Prima parola dopo ">"
                sequence = ""
            
            else:
                # Accumula la sequenza di lettere
                sequence += line
        
        # Salva l'ultima sequenza
        if identifier and sequence:
            results.append((identifier, sequence))
    
    return results

# Usa il percorso al tuo file .fasta
file_path = 'orf_trans_all.fasta'
parsed_data = parse_fasta(file_path)
 
# we create a table   
proteins = pd.DataFrame(parsed_data, columns=["Identifier", "Sequence"])
print(proteins.shape)


(6723, 2)


Trying to map input (sequence) and output (location): checking missing variables

In [114]:
# Check correspondence input-output (with identifiers)
vec = static_data["yORF"].isin(proteins["Identifier"]).astype(int).values
print(vec.shape)
print(np.sum(vec))
vec_not_present = static_data.loc[vec == 0, "yORF"].values
print(vec_not_present)

(6234,)
6212
['YAR044W' 'YBR100W' 'YDR474C' 'YFL006W' 'YFR024C' 'YGL046W' 'YGR272C'
 'YJL012C-A' 'YJL017W' 'YJL018W' 'YJL021C' 'YPR090W' 'YAR061W' 'YAR062W'
 'YBR075W' 'YDL038C' 'YER186W-A' 'YFL043C' 'YIR044C' 'YKL003W-A' 'YKL199C'
 'YOR088W']


In [115]:
# Merge static_data with proteins to add the protein sequences
static_data_with_sequences = static_data.merge(proteins, left_on='yORF', right_on='Identifier', how='left')

# Drop the redundant 'Identifier' column
static_data_with_sequences.drop(columns=['Identifier'], inplace=True)

# Rename the 'Sequence' column to 'Protein Sequence'
static_data_with_sequences.rename(columns={'Sequence': 'Protein Sequence'}, inplace=True)

# Display the updated dataframe
static_data_with_sequences

,orfid,yORF,gene name,GFP tagged,GFP visualized?,TAP visualized?,abundance,error,localization summary,ambiguous,...,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud,Protein Sequence
0,206,YAL001C,TFC3,tagged,visualized,TAP visualized,125,NaN,"cytoplasm,nucleus",F,...,F,F,F,T,F,F,F,T,F,MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1,5281,YAL002W,VPS8,tagged,visualized,TAP visualized,736,NaN,endosome,F,...,F,F,F,F,F,F,F,F,F,MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2,1073,YAL005C,SSA1,tagged,visualized,TAP visualized,269000,NaN,cytoplasm,F,...,F,F,F,T,F,F,F,F,F,MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3,6096,YAL007C,ERP2,tagged,visualized,TAP visualized,26300,NaN,ER,F,...,F,F,F,F,F,F,F,F,F,MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4,3882,YAL008W,FUN14,tagged,visualized,TAP visualized,1920,NaN,mitochondrion,F,...,F,F,F,F,F,F,F,F,F,MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,4500,YPR200C,ARR2,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,MVSFITSRQLKGLIENQRKDFQVVDLRREDFARDHITNAWHVPVTA...
6230,5814,YPR201W,ARR3,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,MSEDQKSENSVPSKVNMVNRTDILTTIKSLSWLDLMLPFTIILSII...
6231,3172,YPR202W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,MEIENEQICTCIAQILHLLNSLIITFSDDDKTETGQSFVYIDGFLV...
6232,5012,YPR203W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN,F,...,F,F,F,F,F,F,F,F,F,MRTFTDFVSGAPIVRSLQKSTIRKYGYNLAPHMFLLLHVDELSIFS...


In [116]:
# Drop proteins without location (i.e. with all falses and localisation_summary nan)
static_data_sq = static_data_with_sequences.dropna(subset=['localization summary'])

We delete the irrelevant columns:

In [117]:
static_data_sq = static_data_sq.iloc[:, [1] + list(range(9, static_data_with_sequences.shape[1]))]
static_data_sq

,yORF,ambiguous,mitochondrion,vacuole,spindle pole,cell periphery,punctate composite,vacuolar membrane,ER,nuclear periphery,...,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud,Protein Sequence
0,YAL001C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1,YAL002W,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2,YAL005C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,F,F,MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3,YAL007C,F,F,F,F,F,F,F,T,F,...,F,F,F,F,F,F,F,F,F,MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4,YAL008W,F,T,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,YPR187W,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MSDYEEAFNDGNENFEDFDVEHFSDEETYEEKPQFKDGETTDANGK...
4156,YPR188C,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MDHSESLTFNQLTQDYINKLKDAFQMLDEDEDGLISRGDLTKIYAT...
4157,YPR189W,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,F,F,MSDIKQLLKEAKQELTNRDYEETIEISEKVLKLDPDNYFAHIFLGK...
4158,YPR190C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MDELLGEALSAENQTGESTVESEKLVTPEDVMTISSLEQRTLNPDL...


In [118]:
# Drop proteins without sequence
static_data_sq = static_data_sq.dropna(subset=['Protein Sequence'])

In [119]:
static_data_sq

,yORF,ambiguous,mitochondrion,vacuole,spindle pole,cell periphery,punctate composite,vacuolar membrane,ER,nuclear periphery,...,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud,Protein Sequence
0,YAL001C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1,YAL002W,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2,YAL005C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,F,F,MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3,YAL007C,F,F,F,F,F,F,F,T,F,...,F,F,F,F,F,F,F,F,F,MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4,YAL008W,F,T,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,YPR187W,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MSDYEEAFNDGNENFEDFDVEHFSDEETYEEKPQFKDGETTDANGK...
4156,YPR188C,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,MDHSESLTFNQLTQDYINKLKDAFQMLDEDEDGLISRGDLTKIYAT...
4157,YPR189W,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,F,F,MSDIKQLLKEAKQELTNRDYEETIEISEKVLKLDPDNYFAHIFLGK...
4158,YPR190C,F,F,F,F,F,F,F,F,F,...,F,F,F,T,F,F,F,T,F,MDELLGEALSAENQTGESTVESEKLVTPEDVMTISSLEQRTLNPDL...


In [120]:
# Convert F,T to 0,1
static_data_sq.replace({'F': 0, 'T': 1}, inplace=True)
static_data_sq

/tmp/ipykernel_2282/1930149075.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  static_data_sq.replace({'F': 0, 'T': 1}, inplace=True)
/tmp/ipykernel_2282/1930149075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_data_sq.replace({'F': 0, 'T': 1}, inplace=True)


,yORF,ambiguous,mitochondrion,vacuole,spindle pole,cell periphery,punctate composite,vacuolar membrane,ER,nuclear periphery,...,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud,Protein Sequence
0,YAL001C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1,YAL002W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2,YAL005C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3,YAL007C,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4,YAL008W,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,YPR187W,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MSDYEEAFNDGNENFEDFDVEHFSDEETYEEKPQFKDGETTDANGK...
4156,YPR188C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MDHSESLTFNQLTQDYINKLKDAFQMLDEDEDGLISRGDLTKIYAT...
4157,YPR189W,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,MSDIKQLLKEAKQELTNRDYEETIEISEKVLKLDPDNYFAHIFLGK...
4158,YPR190C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MDELLGEALSAENQTGESTVESEKLVTPEDVMTISSLEQRTLNPDL...


In [121]:
# We need to remove * at the end of each sequence (since our embedding model do not have * in dictionary)
static_data_sq['Protein Sequence'] = static_data_sq['Protein Sequence'].str[:-1]

/tmp/ipykernel_2282/1548369123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_data_sq['Protein Sequence'] = static_data_sq['Protein Sequence'].str[:-1]


In [123]:
# Check if all * have been removed
check_star = static_data_sq['Protein Sequence'].str.contains(r'\*').any()
check_star

np.False_

In [124]:
static_data_sq

,yORF,ambiguous,mitochondrion,vacuole,spindle pole,cell periphery,punctate composite,vacuolar membrane,ER,nuclear periphery,...,peroxisome,actin,nucleolus,cytoplasm,ER to Golgi,early Golgi,lipid particle,nucleus,bud,Protein Sequence
0,YAL001C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1,YAL002W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2,YAL005C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3,YAL007C,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4,YAL008W,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,YPR187W,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MSDYEEAFNDGNENFEDFDVEHFSDEETYEEKPQFKDGETTDANGK...
4156,YPR188C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MDHSESLTFNQLTQDYINKLKDAFQMLDEDEDGLISRGDLTKIYAT...
4157,YPR189W,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,MSDIKQLLKEAKQELTNRDYEETIEISEKVLKLDPDNYFAHIFLGK...
4158,YPR190C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,MDELLGEALSAENQTGESTVESEKLVTPEDVMTISSLEQRTLNPDL...
